In [42]:
import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Input, BatchNormalization, Conv2DTranspose, Reshape, Activation
from keras.models import Model,Sequential
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam
import numpy as np
import tensorflow as tf

In [43]:
## Preprocessing
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Reshape X_train into a 2d vector
x_train = x_train.reshape(x_train.shape[0], -1)

In [44]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_train.shape

(60000, 784)

## Generator

In [45]:
# Generator

def make_generator():
    model = Sequential()
    model.add(Dense(7*7*256, use_bias = False, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size
    
    model.add(Conv2DTranspose(128, (5, 5), strides = (1, 1), padding="same", use_bias = "false"))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(64, (5, 5), strides = (2, 2), padding="same", use_bias = "false"))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(1, (5, 5), strides = (2, 2), padding="same", use_bias = "false"))
    model.add(Activation("tanh"))
    assert model.output_shape == (None, 28, 28, 1)
    
    return model
    

In [46]:
generator = make_generator()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

TypeError: call() got an unexpected keyword argument 'training'

## Discriminator

In [15]:
## Discriminator
def make_discriminator():
    
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides = (2, 2), padding = "same", input_shape=[28, 28, 1], activation = "LeakyRelu"))
    assert model.output_shape == (None, 14, 14, 64)
    model.Dropout(0.4)
    
    model.add(Conv2D(128, (5, 5), strides = (2, 2), padding = "same", activation = "LeakyRelu"))
    assert model.output_shape == (None, 7, 7, 128)
    model.Dropout(0.4)
    
    model.Flatten()
    model.add(Dense(1))
    
    return model

In [ ]:
discriminator = make_discriminator()


In [9]:
## GAN